## Prepare the data

In [1]:
from elasticsearch import Elasticsearch

/Users/avinashsrirama/Documents/GitHub/SemanticSearch/venv3/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
from elasticsearch import Elasticsearch

es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic", "N*toUYVx44I4oLt5Sx4B"),
    ca_certs="/Users/avinashsrirama/Documents/Softwares/elasticsearch-8.18.4/config/certs/http_ca.crt"
)

try:
    print(es.ping())
except Exception as e:
    print("Error:", e)

True


In [3]:
import pandas as pd

df = pd.read_csv("AB_NYC_2019.csv").loc[:200]
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [6]:
df.isna().value_counts()

id     name   host_id  host_name  neighbourhood_group  neighbourhood  latitude  longitude  room_type  price  minimum_nights  number_of_reviews  last_review  reviews_per_month  calculated_host_listings_count  availability_365
False  False  False    False      False                False          False     False      False      False  False           False              False        False              False                           False               201
Name: count, dtype: int64

In [5]:
df.fillna("None", inplace=True)

/var/folders/3w/wrrjgv8n3kb0mj3897nsngbm0000gn/T/ipykernel_18592/3777244692.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("None", inplace=True)


## Convert the releveant filed to vector using BERT model

In [7]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

/Users/avinashsrirama/Documents/GitHub/SemanticSearch/venv3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
import numpy
print(numpy.__version__)

1.26.4


In [9]:
df["DescriptionVector"] = df["name"].apply(lambda x: model.encode(x))

In [10]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,DescriptionVector
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365,"[-0.047521215, 0.030446216, 0.001784149, -0.00..."
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355,"[-0.046900775, 0.06132935, 0.0068995296, 0.015..."
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,None,None,1,365,"[0.00039004316, 0.018310182, -0.0044140336, -0..."
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194,"[-0.06862569, 0.026565073, 0.012019095, -0.043..."
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.1,1,0,"[-0.046022147, 0.0156053705, 0.00833617, 0.015..."


In [11]:
es.ping()

True

## Create new index in ElasticSearch!

In [ ]:
from indexMapping import airbnb_indexMapping

es.indices.create(index="airbnb_properties", mappings=airbnb_indexMapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'airbnb_properties'})

Ingest the data into index

In [14]:
record_list = df.to_dict("records")

In [15]:
for record in record_list:
    try:
        es.index(index="airbnb_properties", document=record, id=record["id"])
    except Exception as e:
        print(e)

BadRequestError(400, 'document_parsing_exception', "[1:290] failed to parse field [last_review] of type [date] in document with id '3647'. Preview of field's value: 'None'")
BadRequestError(400, 'document_parsing_exception', "[1:293] failed to parse field [last_review] of type [date] in document with id '7750'. Preview of field's value: 'None'")
BadRequestError(400, 'document_parsing_exception', "[1:311] failed to parse field [last_review] of type [date] in document with id '8700'. Preview of field's value: 'None'")
BadRequestError(400, 'document_parsing_exception', "[1:288] failed to parse field [last_review] of type [date] in document with id '11452'. Preview of field's value: 'None'")
BadRequestError(400, 'document_parsing_exception', "[1:282] failed to parse field [last_review] of type [date] in document with id '11943'. Preview of field's value: 'None'")
BadRequestError(400, 'document_parsing_exception', "[1:302] failed to parse field [last_review] of type [date] in document with 

In [16]:
es.count(index="airbnb_properties")

ObjectApiResponse({'count': 195, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

## Search the data

In [18]:
input_keyword = "Family friendly apartment"
vector_of_input_keyword = model.encode(input_keyword)

query = {
    "field" : "DescriptionVector",
    "query_vector" : vector_of_input_keyword,
    "k" : 2,
    "num_candidates" : 500, 
}

res = es.knn_search(index="airbnb_properties", knn=query , source=["name","neighbourhood_group"])
res["hits"]["hits"]

/var/folders/3w/wrrjgv8n3kb0mj3897nsngbm0000gn/T/ipykernel_16954/2905327603.py:11: GeneralAvailabilityWarning: This API is in technical preview and may be changed or removed in a future release. Elastic will work to fix any issues, but features in technical preview are not subject to the support SLA of official GA features.
  res = es.knn_search(index="airbnb_properties", knn=query , source=["name","neighbourhood_group"])
/var/folders/3w/wrrjgv8n3kb0mj3897nsngbm0000gn/T/ipykernel_16954/2905327603.py:11: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = es.knn_search(index="airbnb_properties", knn=query , source=["name","neighbourhood_group"])


[{'_index': 'airbnb_properties',
  '_id': '16326',
  '_score': 0.6384403,
  '_source': {'name': 'Comfortable 4-bedroom apt in family house.',
   'neighbourhood_group': 'Brooklyn'}},
 {'_index': 'airbnb_properties',
  '_id': '5203',
  '_score': 0.60744524,
  '_source': {'name': 'Cozy Clean Guest Room - Family Apt',
   'neighbourhood_group': 'Manhattan'}}]